In [13]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Load MNIST data from sklearn's openml
mnist = fetch_openml('mnist_784', version=1)
x, y = mnist['data'], mnist['target']

# Normalize pixel values to the range [0, 1]
x = x / 255.0

# Split into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Convert labels to integers and one-hot encode them
y_train = y_train.astype(int).to_numpy()  # Convert to NumPy array
y_test = y_test.astype(int).to_numpy()

one_hot_encoder = OneHotEncoder(sparse_output=False)
y_train = one_hot_encoder.fit_transform(y_train.reshape(-1, 1))
y_test = one_hot_encoder.transform(y_test.reshape(-1, 1))

# Neural network parameters
input_size = 784       # 28x28 pixels flattened
hidden_size = 64       # Number of neurons in the hidden layer
output_size = 10       # Number of classes (digits 0-9)
learning_rate = 0.01   # Learning rate
epochs = 10            # Number of epochs

# Initialize weights and biases
np.random.seed(42)  # For reproducibility
W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2. / input_size)
b1 = np.zeros((1, hidden_size))
W2 = np.random.randn(hidden_size, output_size) * np.sqrt(2. / hidden_size)
b2 = np.zeros((1, output_size))

# Activation functions
def sigmoid(x):
    x = np.clip(x, -500, 500)
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

# Loss function
def cross_entropy_loss(predictions, targets):
    return -np.mean(np.sum(targets * np.log(predictions + 1e-9), axis=1))

# Forward pass
def forward(X):
    z1 = np.dot(X, W1) + b1
    a1 = sigmoid(z1)
    z2 = np.dot(a1, W2) + b2
    a2 = softmax(z2)
    return z1, a1, z2, a2

# Backward pass
def backward(X, y, z1, a1, z2, a2):
    global W1, b1, W2, b2

    # Output layer error
    error_output = a2 - y
    dW2 = np.dot(a1.T, error_output)
    db2 = np.sum(error_output, axis=0, keepdims=True)

    # Hidden layer error
    error_hidden = np.dot(error_output, W2.T) * sigmoid_derivative(a1)
    dW1 = np.dot(X.T, error_hidden)
    db1 = np.sum(error_hidden, axis=0, keepdims=True)

    # Update weights and biases
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2

# Training loop
for epoch in range(epochs):
    z1, a1, z2, a2 = forward(x_train)
    loss = cross_entropy_loss(a2, y_train)
    backward(x_train, y_train, z1, a1, z2, a2)
    
    if epoch % 1 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}")

# Testing accuracy
_, _, _, a2_test = forward(x_test)
predictions = np.argmax(a2_test, axis=1)
accuracy = np.mean(predictions == np.argmax(y_test, axis=1))
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Epoch 1/50, Loss: 2.4713
Epoch 2/50, Loss: 4.8140
Epoch 3/50, Loss: 3.3925
Epoch 4/50, Loss: 2.2720
Epoch 5/50, Loss: 2.1561
Epoch 6/50, Loss: 2.0228
Epoch 7/50, Loss: 1.8311
Epoch 8/50, Loss: 1.7122
Epoch 9/50, Loss: 1.7189
Epoch 10/50, Loss: 1.9309
Epoch 11/50, Loss: 1.5439
Epoch 12/50, Loss: 1.2936
Epoch 13/50, Loss: 1.4601
Epoch 14/50, Loss: 1.3112
Epoch 15/50, Loss: 1.3715
Epoch 16/50, Loss: 1.3340
Epoch 17/50, Loss: 1.1482
Epoch 18/50, Loss: 0.8838
Epoch 19/50, Loss: 0.9438
Epoch 20/50, Loss: 1.0354
Epoch 21/50, Loss: 1.0274
Epoch 22/50, Loss: 0.7570
Epoch 23/50, Loss: 0.8099
Epoch 24/50, Loss: 0.9842
Epoch 25/50, Loss: 0.9360
Epoch 26/50, Loss: 0.7909
Epoch 27/50, Loss: 0.6327
Epoch 28/50, Loss: 0.6026
Epoch 29/50, Loss: 0.5921
Epoch 30/50, Loss: 0.6213
Epoch 31/50, Loss: 0.7523
Epoch 32/50, Loss: 0.5985
Epoch 33/50, Loss: 0.5793
Epoch 34/50, Loss: 0.5092
Epoch 35/50, Loss: 0.5471
Epoch 36/50, Loss: 0.6018
Epoch 37/50, Loss: 0.7500
Epoch 38/50, Loss: 0.7046
Epoch 39/50, Loss: 0.

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Load and preprocess the MNIST data
mnist = fetch_openml('mnist_784', version=1)
x, y = mnist['data'], mnist['target']

# Normalize pixel values to the range [0, 1]
x = x / 255.0

# Split into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Convert labels to integers and one-hot encode them
y_train = y_train.astype(int).to_numpy()
y_test = y_test.astype(int).to_numpy()

one_hot_encoder = OneHotEncoder(sparse_output=False)
y_train = one_hot_encoder.fit_transform(y_train.reshape(-1, 1))
y_test = one_hot_encoder.transform(y_test.reshape(-1, 1))

# Convert data to PyTorch tensors
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Define the neural network model
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.sigmoid = nn.Sigmoid()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.sigmoid(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

# Initialize the model, loss function, and optimizer
input_size = 784
hidden_size = 64
output_size = 10
learning_rate = 0.01
epochs = 10

model = NeuralNetwork(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    # Forward pass
    outputs = model(x_train_tensor)
    loss = criterion(outputs, torch.argmax(y_train_tensor, dim=1))

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 1 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

# Testing accuracy
with torch.no_grad():
    outputs_test = model(x_test_tensor)
    _, predicted = torch.max(outputs_test, 1)
    accuracy = (predicted == torch.argmax(y_test_tensor, dim=1)).float().mean()
    print(f"Test Accuracy: {accuracy.item() * 100:.2f}%")

ValueError: could not determine the shape of object type 'DataFrame'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Load and preprocess the MNIST data
mnist = fetch_openml('mnist_784', version=1)
x, y = mnist['data'], mnist['target']

# Normalize pixel values to the range [0, 1]
x = x / 255.0

# Split into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Convert labels to integers and one-hot encode them
y_train = y_train.astype(int).to_numpy()
y_test = y_test.astype(int).to_numpy()

one_hot_encoder = OneHotEncoder(sparse_output=False)
y_train = one_hot_encoder.fit_transform(y_train.reshape(-1, 1))
y_test = one_hot_encoder.transform(y_test.reshape(-1, 1))

# Convert data to NumPy arrays before creating PyTorch tensors
x_train_tensor = torch.tensor(x_train.to_numpy(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test.to_numpy(), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Define the neural network model
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.sigmoid = nn.Sigmoid()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.sigmoid(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

# Initialize the model, loss function, and optimizer
input_size = 784
hidden_size = 64
output_size = 10
learning_rate = 0.01
epochs = 10

model = NeuralNetwork(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    # Forward pass
    outputs = model(x_train_tensor)
    loss = criterion(outputs, torch.argmax(y_train_tensor, dim=1))

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 1 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

# Testing accuracy
with torch.no_grad():
    outputs_test = model(x_test_tensor)
    _, predicted = torch.max(outputs_test, 1)
    accuracy = (predicted == torch.argmax(y_test_tensor, dim=1)).float().mean()
    print(f"Test Accuracy: {accuracy.item() * 100:.2f}%")